In [6]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.models import load_model
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import dill

In [2]:
# Cargar los datos
with open('test_data.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

In [3]:
# Cargar el modelo LSTM
model_lstm = load_model('modelo_lstm.keras')

# Cargar el modelo RNN
model_rnn = load_model('modelo_rnn.keras')

C:\Users\User\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:562: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
# Redefinir el modelo basado en reglas
# Cargar el modelo basado en reglas
with open('modelo_reglas.pkl', 'rb') as file:
    rule_based_model = dill.load(file)

In [13]:
# Cargar el modelo de Transformers (Hugging Face)
model_checkpoint = "Ernesto-1997/roberta-base-bne-finetuned-spanish_sarcastic_texts"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model_transformers = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

In [9]:
# Predicción con LSTM
def predict_lstm(X_test):
    return (model_lstm.predict(X_test) > 0.5).astype("int32")

# Predicción con RNN
def predict_rnn(X_test):
    return (model_rnn.predict(X_test) > 0.5).astype("int32")

In [10]:
# Predicción con el modelo basado en reglas
def predict_rules(X_test):
    return np.array([rule_based_model(text) for text in X_test])


In [11]:
# Predicción con el modelo Transformers de Hugging Face
def predict_transformers(X_test):
    inputs = tokenizer(X_test, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        logits = model_transformers(**inputs).logits
    return (torch.sigmoid(logits) > 0.5).int().numpy()

In [12]:
# Evaluar los modelos y calcular las métricas
def evaluar_modelo(y_true, y_pred, nombre_modelo):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    print(f"Resultados para {nombre_modelo}:")
    print(f"Precisión: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print()

In [14]:
# Realizar las predicciones para cada modelo
y_pred_lstm = predict_lstm(X_test)
y_pred_rnn = predict_rnn(X_test)

120/120 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [19]:
y_pred_rules = predict_rules(X_test)

C:\Users\User\AppData\Local\Temp\ipykernel_16656\2232016145.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [16]:
y_pred_transformers = predict_transformers(X_test)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [20]:
evaluar_modelo(y_test, y_pred_lstm, "LSTM")
evaluar_modelo(y_test, y_pred_rnn, "RNN")
evaluar_modelo(y_test, y_pred_rules, "Modelo basado en reglas")

Resultados para LSTM:
Precisión: 0.7810760667903525
Recall: 0.8425617078052035
F1-Score: 0.8106546854942233
Accuracy: 0.8455497382198953

Resultados para RNN:
Precisión: 0.7358133669609079
Recall: 0.7785190126751167
F1-Score: 0.7565640194489465
Accuracy: 0.8034031413612566

Resultados para Modelo basado en reglas:
Precisión: 0.0
Recall: 0.0
F1-Score: 0.0
Accuracy: 0.6075916230366493



C:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# Evaluar cada modelo
evaluar_modelo(y_test, y_pred_transformers, "Transformers (Hugging Face)")

120/120 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


C:\Users\User\AppData\Local\Temp\ipykernel_1112\3263194373.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return 1 if any(word in text for word in sarcasm_words) else 0


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).